<a href="https://colab.research.google.com/github/klvntagoe/DesneNets/blob/master/DenseNet_121_for_learning_rate_experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This cell imports the drive library and mounts your Google Drive as a VM local drive. 
# You can access to your Drive files using this path “/content/gdrive/My Drive/”
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import math
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time, copy, os
import torchvision.transforms as transforms
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [0]:
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")


model_name = "densenet"
num_classes = 10            #CIFAR10
batch_size = 64
epochs = 20
learning_rate = 0.0001
pretrained = False           #When False, we train a model from scratch, when True we initialize weights to be a pretrained model on ImageNet
feature_extract = False     #When False, we finetune the whole model, when True we only update the reshaped layer params

#NOTE feature_extract IS FALSE RN
#REMEMBER TO CHANGE RESULTS FILE NAME WHEN CHANGING EXPERIMENTATION VALUES
data_dir = "./data"
save = './gdrive/My Drive/results_learning_rate=' + str(learning_rate) + '_from_scratch.csv'

print('DenseNet 121 for CIFAR10:')
print('batch size = ' + str(batch_size))
print('Epochs = ' + str(epochs))
print('Learning rate = ' + str(learning_rate))

DenseNet 121 for CIFAR10:
batch size = 64
Epochs = 20
Learning rate = 0.0001


In [0]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


def initialize_model(num_classes, feature_extract, use_pretrained):
    model_ft = models.densenet121(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.classifier.in_features
    model_ft.classifier = nn.Linear(num_ftrs, num_classes)
    input_size = 224
    return model_ft, input_size

In [0]:
#INITIALIZE MODEL
model_ft, input_size = initialize_model(num_classes, feature_extract, pretrained)



#GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.device_count() > 0:
  print(torch.cuda.device_count(), "GPUs available")
  model_ft = nn.DataParallel(model_ft)
model_ft.to(device)



#LOAD DATA
transform = transforms.Compose(
    [transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
    )
trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=False, transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



#OPTIMIZATION
params_to_update = model_ft.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

1 GPUs available
Files already downloaded and verified
Params to learn:
	 module.features.conv0.weight
	 module.features.norm0.weight
	 module.features.norm0.bias
	 module.features.denseblock1.denselayer1.norm1.weight
	 module.features.denseblock1.denselayer1.norm1.bias
	 module.features.denseblock1.denselayer1.conv1.weight
	 module.features.denseblock1.denselayer1.norm2.weight
	 module.features.denseblock1.denselayer1.norm2.bias
	 module.features.denseblock1.denselayer1.conv2.weight
	 module.features.denseblock1.denselayer2.norm1.weight
	 module.features.denseblock1.denselayer2.norm1.bias
	 module.features.denseblock1.denselayer2.conv1.weight
	 module.features.denseblock1.denselayer2.norm2.weight
	 module.features.denseblock1.denselayer2.norm2.bias
	 module.features.denseblock1.denselayer2.conv2.weight
	 module.features.denseblock1.denselayer3.norm1.weight
	 module.features.denseblock1.denselayer3.norm1.bias
	 module.features.denseblock1.denselayer3.conv1.weight
	 module.features.dens

In [0]:
#TRAINING
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params_to_update,lr=learning_rate)

# Start log
with open(save, 'w') as f:
  f.write('epoch,train_loss,valid_loss,validation_accuracy,\n')

for epoch in range(epochs):
    running_loss = 0
    i = 0     #Peace of mind
    j = 0     #Peace of mind
    for images,labels in train_loader:
        if use_cuda:
            images, labels = images.cuda(), labels.cuda()
        train = Variable(images.view(-1,3,224,224))
        labels = Variable(labels)
        
        optimizer.zero_grad()
        
        output = model_ft(train)
        loss = criterion(output,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if (i % 100 == 0): print(i)
        i+=1
        
    else:
        test_loss = 0
        accuracy = 0

        with torch.no_grad(): #Turning off gradients to speed up
            model_ft.eval()
            for images,labels in test_loader:
                if use_cuda:
                    images, labels = images.cuda(), labels.cuda()
                test = Variable(images.view(-1,3,224,224))
                labels = Variable(labels)
                
                log_ps = model_ft(test)
                test_loss += criterion(log_ps,labels)
            
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim = 1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
                if (j % 100 == 0):print(j)
                j+=1
        model_ft.train() 
        
        train_l = running_loss/len(train_loader)
        test_l = test_loss/len(test_loader)
        acc = accuracy/len(test_loader)
        
        # Print results
        print("Epoch: {}/{}.. ".format(epoch+1, epochs),
            "Training Loss: {:.3f}.. ".format(train_l),
            "Test Loss: {:.3f}.. ".format(test_l),
            "Test Accuracy: {:.3f}".format(acc)
             )
        
        # Log results
        with open(save, 'a') as f:
          f.write('%03d,%0.6f,%0.6f,%0.5f,\n' % (
                (epoch + 1),
                train_l,
                test_l,
                acc,
            ))

0
100
200
300
400
500
600
700
0
100
Epoch: 1/20..  Training Loss: 1.418..  Test Loss: 1.131..  Test Accuracy: 0.591
0
100
200
300
400
500
600
700
0
100
Epoch: 2/20..  Training Loss: 0.982..  Test Loss: 1.002..  Test Accuracy: 0.649
0
100
200
300
400
500
600
700
0
100
Epoch: 3/20..  Training Loss: 0.763..  Test Loss: 0.765..  Test Accuracy: 0.733
0
100
200
300
400
500
600
700
0
100
Epoch: 4/20..  Training Loss: 0.602..  Test Loss: 0.701..  Test Accuracy: 0.755
0
100
200
300
400
500
600
700
0
100
Epoch: 5/20..  Training Loss: 0.486..  Test Loss: 0.648..  Test Accuracy: 0.784
0
100
200
300
400
500
600
700
0
100
Epoch: 6/20..  Training Loss: 0.387..  Test Loss: 0.758..  Test Accuracy: 0.757
0
100
200
300
400
500
600
700
0
100
Epoch: 7/20..  Training Loss: 0.313..  Test Loss: 0.709..  Test Accuracy: 0.771
0
100
200
300
400
500
600
700
0
100
Epoch: 8/20..  Training Loss: 0.244..  Test Loss: 0.625..  Test Accuracy: 0.800
0
100
200
300
400
500
600
700
0
100
Epoch: 9/20..  Training Loss: 0.189.

In [0]:
torch.save(model_ft.state_dict(), "./gdrive/My Drive/denseNet121_lr="+ str(learning_rate) + "_from_scratch.dat")